In [23]:

def sharpen_image(image):
        # Define a sharpening kernel
    kernel = np.array([[0, -1, 0],
                        [-1, 5, -1],
                        [0, -1, 0]])
        
        # Apply the kernel using filter2D
    sharpened = cv2.filter2D(image, -1, kernel)
        
    return sharpened
    
def sketch(image):
        # Load the image in grayscale 
        # Step 1: Invert the grayscale image (negative of the image)
    inverted_image = cv2.bitwise_not(image)
        
        # Step 2: Apply a Gaussian blur to the inverted image
    blurred_image = cv2.GaussianBlur(inverted_image, (21, 21), 0)
        
        # Step 3: Invert the blurred image
    inverted_blurred = cv2.bitwise_not(blurred_image)
        
        # Step 4: Create the sketch effect by dividing the original image by the inverted blurred image
    sketch = cv2.divide(image, inverted_blurred, scale=256.0)
        
    sketch=sharpen_image(sketch)
        # Step 5: Reduce exposure in the final sketch
    exposure_factor = 0.5  # Adjust this factor (between 0 and 1) to reduce exposure
    sketch = np.clip(sketch * exposure_factor, 0, 255).astype(np.uint8)
    sketch=cv2.cvtColor(sketch, cv2.COLOR_BGR2GRAY)
    return sketch
    
            


In [25]:
def fil1(face, frame):
    x, y, w, h=face
    x1=x+int(2*(w/6))
    y1=y+int(2*(h/6)+10)
    w1=int(w/3)
    h1=int(h/3)
    
                    #frame=cv2.rectangle(frame, (x,y), (x+w, y+h), (255, 0, 0), 3)
    k=cv2.imread(r"C:\Users\joshi\Documents\EDGE AI\Face_Detection\Images\pig.png", cv2.IMREAD_UNCHANGED)
    k1=cv2.resize(k, (w1,h1))  
    
    bgr=k1[:, :, :3]  #BGR channels
    alpha=k1[:,:,3]/255.0 #Alpha channel normalized to 0-1
                
                        #Extracting the region of interest from the frame
    roi=frame[y1:y1+h1, x1:x1+h1]
                
                        #Blend the png image and the roi using alpha channel
                
    for c in range(0,3):
        roi[:, :,c]=roi[:,:,c]*(1-alpha)+bgr[:,:,c]*alpha
                
                        #Place the blended image back into the frame
                        
                        #p.append(x,y,w,h)
                        
    frame[y1:y1+h1, x1:x1+w1]=roi
    return frame


def filter(val, frame):
    import cv2
    import os
    import numpy as np
    face_cascade = cv2.CascadeClassifier("C:/Users/joshi/Documents/EDGE AI/Face_Detection/haarcascade_frontalface_default.xml")
    gray_image=cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    detections=face_cascade.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=6)
    for face in detections: 
        if (val==1):
            frame=fil1(face, frame)

    return frame
        

In [27]:
def portrait(frame):
    import mediapipe as mp
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose()
    mp_face_detection = mp.solutions.face_detection
    face_detection = mp_face_detection.FaceDetection()
    
    # Function to expand landmark points (more in x-direction)
    def expand_points_x_direction(points, width, height, x_offset=50, y_offset=10):
        expanded_points = []
        for (x, y) in points:
            expanded_x_left = np.clip(x - x_offset + 30, 0, width)
            expanded_x_right = np.clip(x + x_offset - 30, 0, width)
            expanded_y_up = np.clip(y - y_offset - 100, 0, height)
            expanded_y_down = np.clip(y + y_offset, 0, height)
            
            expanded_points.append((expanded_x_left, expanded_y_up))
            expanded_points.append((expanded_x_right, expanded_y_down))
        
        return expanded_points
    
    
        # Convert the frame to RGB for MediaPipe processing
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
        # Detect human pose
    pose_result = pose.process(rgb_frame)
        
        # Detect face in the frame
    face_result = face_detection.process(rgb_frame)
    
        # Create a mask for the human body
    mask = np.zeros_like(frame, dtype=np.uint8)
    
    height, width, _ = frame.shape
    
        # If body landmarks are detected
    if pose_result.pose_landmarks:
            # Collect all body landmark points
        body_points = []
        for landmark in pose_result.pose_landmarks.landmark:
            x = int(landmark.x * width)
            y = int(landmark.y * height)
            body_points.append((x, y))
    
            # Expand the points (more expansion in x-direction)
        expanded_body_points = expand_points_x_direction(body_points, width, height, x_offset=50, y_offset=30)
    
            # Convert the points into a numpy array
        body_points = np.array(expanded_body_points)
    
            # Draw the convex hull around the expanded body points
        if len(body_points) > 0:
            hull = cv2.convexHull(body_points)
    
                # Fill the convex hull on the mask
            cv2.fillConvexPoly(mask, hull, (255, 255, 255))
    
        # If face landmarks are detected, exclude the face from being blurred
    if face_result.detections:
        for detection in face_result.detections:
            bboxC = detection.location_data.relative_bounding_box
            x_min = int(bboxC.xmin * width)
            y_min = int(bboxC.ymin * height)
            w = int(bboxC.width * width)
            h = int(bboxC.height * height)
    
                # Draw a white rectangle (mask) over the face region
            cv2.rectangle(mask, (x_min, y_min), (x_min + w, y_min + h), (255, 255, 255), -1)
    
        # Dilate the mask to ensure it covers all areas around the body
    kernel = np.ones((15, 15), np.uint8)  # Dilation kernel size
    dilated_mask = cv2.dilate(mask, kernel, iterations=1)
    
        # Convert the mask to grayscale
    gray_mask = cv2.cvtColor(dilated_mask, cv2.COLOR_BGR2GRAY)
    
        # Create a binary mask (human body and face as white, background as black)
    _, binary_mask = cv2.threshold(gray_mask, 1, 255, cv2.THRESH_BINARY)
    
        # Invert the mask to blur only the background
    inverted_mask = cv2.bitwise_not(binary_mask)
    
        # Blur the background
    blurred_frame = cv2.GaussianBlur(frame, (21, 21), 30)
    
        # Combine the blurred background with the original body and face (excluding hair blur)
    foreground = cv2.bitwise_and(frame, frame, mask=binary_mask)
    background = cv2.bitwise_and(blurred_frame, blurred_frame, mask=inverted_mask)
    
        # Combine foreground and background
    result_frame = cv2.add(foreground, background)
    return result_frame

In [ ]:
import tkinter as tk
from tkinter import filedialog
from tkinter import ttk
import cv2
from PIL import Image, ImageTk
import numpy as np

class PhotoEditorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Photo Editor")
        self.root.geometry("800x600")

        self.video_source = 0
        self.vid = cv2.VideoCapture(self.video_source)
        
        self.canvas = tk.Canvas(self.root, width=640, height=480)
        self.canvas.pack()

        self.filter_option = tk.StringVar(value="None")
        self.show_video = True  # Flag to toggle between video feed and uploaded image
        self.uploaded_image = None  # Placeholder for the uploaded image

        # Control buttons
        self.create_controls()

        self.update_frame()

    def create_controls(self):
        control_frame = tk.Frame(self.root)
        control_frame.pack(side=tk.TOP, pady=10)

        btn_capture = ttk.Button(control_frame, text="Capture Photo", command=self.capture_photo)
        btn_capture.grid(row=0, column=0, padx=10)

        btn_upload = ttk.Button(control_frame, text="Upload Photo", command=self.upload_photo)
        btn_upload.grid(row=0, column=1, padx=10)

        filter_label = ttk.Label(control_frame, text="Select Filter:")
        filter_label.grid(row=0, column=2, padx=10)

        filter_menu = ttk.OptionMenu(control_frame, self.filter_option, "None", "Sketch", "Piggy", "Portrait")
        filter_menu.grid(row=0, column=3, padx=10)

        btn_magic_eraser = ttk.Button(control_frame, text="Magic Eraser", command=self.magic_eraser_tool)
        btn_magic_eraser.grid(row=0, column=4, padx=10)

        btn_show_video = ttk.Button(control_frame, text="Show Video", command=self.show_video_feed)
        btn_show_video.grid(row=0, column=5, padx=10)

    def update_frame(self):
        global frame1
        # If the flag is set to show video, capture and display the video feed
        if self.show_video:
            ret, frame = self.vid.read()
            if ret:
                frame = self.apply_filter(frame)
                frame1=frame
                self.photo = ImageTk.PhotoImage(image=Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)))
                self.canvas.create_image(0, 0, image=self.photo, anchor=tk.NW)
        else:
            # If an image is uploaded, display the uploaded image
            if self.uploaded_image is not None:
                img = cv2.cvtColor(np.array(self.uploaded_image), cv2.COLOR_RGB2BGR)
                img = self.apply_filter(img)
                frame1=img
                self.photo = ImageTk.PhotoImage(image=Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)))
                self.canvas.create_image(0, 0, image=self.photo, anchor=tk.NW)

        self.root.after(10, self.update_frame)

    def apply_filter(self, frame):
        filter_choice = self.filter_option.get()
        if filter_choice == "Sketch":
            return sketch(frame)
        elif filter_choice == "Piggy":
            return filter(1,frame)
        elif filter_choice == "Portrait":
            return portrait(frame)
        return frame

 
    def capture_photo(self):
        if self.show_video:
            ret = self.vid.read()
            if ret:
                cv2.imwrite("captured_image.jpg", frame1)
                print("Photo saved!")

        else:
            cv2.imwrite("captured_image.jpg", frame1)
            print("Photo saved!")

    def upload_photo(self):
        file_path = filedialog.askopenfilename(filetypes=[("Image Files", ".jpg;.jpeg;*.png")])
        if file_path:
            img = Image.open(file_path)
            img = img.resize((640, 480))  # Resize to fit canvas size
            self.uploaded_image = img
            self.show_video = False  # Stop showing video when an image is uploaded

    def magic_eraser_tool(self):
        #yet to implement
        print("NULL")

    def show_video_feed(self):
        # Re-enable video feed and hide the uploaded image
        self.show_video = True

    def __del__(self):
        if self.vid.isOpened():
            self.vid.release()

# Run the application
if __name__ == "__main__":
    root = tk.Tk()
    app = PhotoEditorApp(root)
    root.mainloop()


C:\Users\joshi\OneDrive\Documents\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


Photo saved!
